# Single Slide Processing Pipeline
### Generate ASAP Annotations XML file as result

In [8]:
import numpy as np
from PIL import Image
from utils import *
from ASAPAnnotationXML import *
from keras.models import Model, load_model
import openslide
from math import ceil
from scipy.ndimage.measurements import label
from scipy.spatial.distance import pdist, squareform
import cv2

In [10]:
def RunModelOnTiles(modelfile, tile_list, level=2):
    model = load_model(modelfile)
    batch_size = 64
    steps = math.ceil((tile_list.shape[0])/batch_size)
    gen = patch_batch_generator(slide, tile_list, batch_size, level=level)
    
    predictions = model.predict_generator(gen, steps, verbose=1)
    
    return predictions

def markImage(slide, tile_list, predictions, return_mask=False, from_level=2, img_level=6, tile_size=256, threshold=0.4, color=[255,0,0]):
    if return_mask:
        img_shape = (slide.level_dimensions[img_level][1], slide.level_dimensions[img_level][1])
        img = np.zeros(img_shape)
        color = 255
    else:
        img = getRegionFromSlide(slide, level=img_level)
    tile = ceil(tile_size * slide.level_downsamples[from_level]/slide.level_downsamples[img_level])
    for i in range(len(tile_list)):
        if predictions[i,1] >= threshold:
            coord = ( tile_list[i][0] , tile_list[i][1] )
            img[coord[1]:coord[1]+tile, coord[0]:coord[0]+tile] = color
    return img

def getDetectedMaskList(mask):
    labels, detections = label(mask)
    mask_list = []
    for i in range(1, detections+1):
        m = np.zeros_like(mask)
        m[labels==i] = 255
        mask_list.append(m)
    return mask_list
        
def getBoundaryList(mask):
    mask_list = getDetectedMaskList(mask)
    boundary_list = []
    kernel = np.ones((2,2), dtype=np.uint8)
    for m in mask_list:
        grad = cv2.morphologyEx(m, cv2.MORPH_GRADIENT, kernel)
        boundary_list.append(grad)
    return boundary_list

def getBorderPointsList(mask):
    boundary_list = getBoundaryList(mask)
    borderpoints_list = []
    for b in boundary_list:
        borderpoints_list.append(np.argwhere(b))
    return borderpoints_list

def getAnnotation(borderpoints, downsample_rate=3):
    pairwise_distance = squareform(pdist(borderpoints))
    max_dist = np.max(pairwise_distance) + 100
    
    for i in range(pairwise_distance.shape[0]):
        pairwise_distance[i,i] = max_dist
    
    seq = [0]
    for i in range(pairwise_distance.shape[0]):
        seq.append(np.argmin(pairwise_distance[seq[-1]]))
        pairwise_distance[seq[-1], seq[-2]] = max_dist
    
    annotation = []
    for s in seq[::downsample_rate]:
        annotation.append(borderpoints[s])
        
    return np.array(annotation)
    
def getAnnotationsList(mask, downsample_rate=3):
    borderpoints_list = getBorderPointsList(mask)
    
    annotation_list = []
    for b in borderpoints_list:
        annotation_list.append(getAnnotation(b, downsample_rate))
        
    return annotation_list

### Get tile list for slide

In [2]:
slide = getWSI('../jupyter_notebooks/patient_015/patient_015_node_1.tif')
tile_list = getTileList(slide)

### Load and run model on tiles and return predictions